In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
from sklearn.metrics import f1_score

In [3]:
d = pd.DataFrame(np.random.rand(10,10))
d

,0,1,2,3,4,5,6,7,8,9
0,0.466670,0.945122,0.344798,0.034212,0.639863,0.146643,0.037400,0.579928,0.929524,0.855421
1,0.801363,0.541243,0.048138,0.336229,0.841224,0.099304,0.505761,0.443233,0.578308,0.421292
2,0.433254,0.488747,0.185712,0.269447,0.038266,0.686622,0.918437,0.942568,0.885490,0.158104
3,0.974435,0.341683,0.565098,0.902806,0.161330,0.723748,0.978501,0.411214,0.756938,0.150865
4,0.486044,0.182729,0.151888,0.665074,0.486503,0.696100,0.607939,0.682866,0.723217,0.291004
5,0.535661,0.785412,0.168049,0.254804,0.189914,0.559956,0.688751,0.947861,0.370087,0.358290
6,0.476866,0.833050,0.938731,0.489034,0.620878,0.712349,0.163763,0.184329,0.507809,0.083123
7,0.216532,0.180753,0.278110,0.486865,0.152070,0.760763,0.271813,0.723306,0.868529,0.485392
8,0.242191,0.218225,0.674348,0.092501,0.466424,0.185134,0.415377,0.220856,0.772891,0.537820
9,0.783670,0.108677,0.544188,0.661202,0.231140,0.768422,0.915103,0.917298,0.683283,0.085774


In [4]:
d.agg({"a": lambda x: (x**2).sum()})

/home/joe/kaggle/mercedes/.env/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


KeyError: 'a'

In [2]:
def f1_score_single(y_true, y_pred):
    try:
        y_true = set(list(y_true))
        y_pred = set(list(y_pred))
        cross_size = len(y_true & y_pred)
        if cross_size == 0: return 0.
        p = 1. * cross_size / len(y_pred)
        r = 1. * cross_size / len(y_true)
        return 2 * p * r / (p + r)
    except:
        return 1
    
def f1_score(y_true, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

In [3]:
orders = pd.read_csv("../data/orders.csv", index_col=0)
aisles = pd.read_csv("../data/aisles.csv", index_col=0)
deps = pd.read_csv("../data/departments.csv", index_col=0)
prior = pd.read_csv("../data/order_products__prior.csv", index_col=0)
prior = prior[prior["reordered"]==True]
train = pd.read_csv("../data/order_products__train.csv", index_col=0)
train = prior[prior["reordered"]==True]
prods = pd.read_csv("../data/products.csv", index_col=0)

/home/joe/kaggle/mercedes/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
reordered = prior["product_id"].groupby(level=0).apply(list)

In [5]:
targets = train["product_id"].groupby(level=0).apply(list)

In [6]:
targets = orders[["user_id"]].join(targets, how="right")

In [7]:
users = targets.user_id

In [8]:
last_orders = orders[orders["eval_set"] == "prior"].groupby("user_id").apply(lambda x: x.index[-1])
last_orders.name = "last_order"

In [9]:
targets = targets.join(last_orders, on="user_id", how="left")

In [15]:
df = targets.join(reordered, on="last_order", rsuffix="_pred", how="left")

In [16]:
df["scores"] = [f1_score_single(true, pred) for true, pred in zip(df["product_id"].values, df["product_id_pred"].values)]

In [40]:
test_last_orders = orders[orders["eval_set"] != "test"].groupby("user_id").apply(lambda x: x.index[-1])
test_last_orders.name = "last_order"

In [42]:
samp = orders[["user_id"]].join(samp, how="right")
samp = samp.join(last_orders, on="user_id", how="left")

In [52]:
sub1 = train.loc[samp.last_order].groupby(level=0)["product_id"].apply(lambda x: " ".join(str(a) for a in x))

In [54]:
sub = samp.join(sub1, on="last_order", how="left")

In [60]:
sub["products"] = sub["product_id"].str.replace("nan", "None").str.replace(".0", "")

In [62]:
sub[["products"]].to_csv("sub1.csv.gz", compression="gzip")

In [65]:
samp.loc[329954]

user_id                 4
products      39276 29259
last_order        2557754
Name: 329954, dtype: object

In [71]:
train[train.index==2557754]

,product_id,add_to_cart_order,reordered
order_id,,,


In [26]:
sub = df["product_id"].apply(lambda x: " ".join(str(a) for a in x))

In [27]:
samp = pd.read_csv("../data/sample_submission.csv", index_col=0)

In [33]:
samp.tail()

,products
order_id,
3420740,39276 29259
3420877,39276 29259
3420888,39276 29259
3420989,39276 29259
3421054,39276 29259


In [29]:
sub.head()

order_id
2                   33120 28985 45918 17794 40141 1819
3      33754 24838 17704 21903 17668 46667 17461 32665
4    26434 39758 27761 10054 21351 22598 34862 4028...
5    13176 15005 47329 27966 23909 48370 13245 9633...
8                                                23423
Name: product_id, dtype: object